In [19]:
import mysql.connector
import torch.nn.functional as F
import numpy as np

In [23]:
import torch
def state2tensor(state_str):
    state_list = list(map(float, state_str.replace("\n", "").strip('[]').split()))
    # 将列表转换为 NumPy 数组
    state_array = np.array(state_list)
    # 将 NumPy 数组转换为 PyTorch 张量
    state_tensor = torch.tensor(state_array)
    return state_tensor

In [34]:
student_id = 53167
try:
        # 连接到 MySQL 数据库
    conn = mysql.connector.connect(
            host="mysql.mysql",
            user="root",
            password="pYRGObpCdG",
            database="sage_javon",
            port=3306
        )

    if conn.is_connected():
            print('Connected to MySQL database')

        # 创建游标对象，用于执行查询
    cursor = conn.cursor(dictionary=True)

        # 获取指定学生的知识状态
    query = "SELECT knowledge_state FROM student WHERE id = %s"
    cursor.execute(query, (student_id,))
    student_knowledge_state_row = cursor.fetchone()

    if not student_knowledge_state_row:
        raise ValueError(f"No student found with ID {student_id}")
    
    student_knowledge_state = student_knowledge_state_row['knowledge_state']

        # 获取其他所有学生的知识状态
    query = "SELECT id, knowledge_state FROM student WHERE id != %s"
    cursor.execute(query, (student_id,))
    other_students = cursor.fetchall()

    similarity_dict = {}

    threshold = 0.90 # 设置相似度阈值

    for other_student in other_students:
        other_student_id = other_student['id']
        other_student_knowledge_state = other_student['knowledge_state']

        if other_student_knowledge_state != None:
            # 计算二者的余弦相似度
            similarity = F.cosine_similarity(
                state2tensor(student_knowledge_state),
                state2tensor(other_student_knowledge_state),
                dim=0
            )
            if similarity > threshold:
                similarity_dict[other_student_id] = similarity
        # 将学生 ID 和相似度保存到字典 similarity_dict 中

    # 相似度降序
    sorted_similar_students = sorted(similarity_dict.items(), key=lambda x: x[1], reverse=True)

    print(sorted_similar_students )

except mysql.connector.Error as e:
    print("Error connecting to MySQL database:", e)

finally:
    # 关闭游标和数据库连接
    if 'cursor' in locals() and cursor:
        cursor.close()
    if 'conn' in locals() and conn.is_connected():
        conn.close()
        print('MySQL connection closed')

Connected to MySQL database
[('79768', tensor(0.9755, dtype=torch.float64)), ('78367', tensor(0.9739, dtype=torch.float64)), ('78280', tensor(0.9739, dtype=torch.float64)), ('78729', tensor(0.9693, dtype=torch.float64)), ('96229', tensor(0.9593, dtype=torch.float64)), ('78983', tensor(0.9526, dtype=torch.float64)), ('78920', tensor(0.9526, dtype=torch.float64)), ('78718', tensor(0.9526, dtype=torch.float64)), ('78842', tensor(0.9517, dtype=torch.float64)), ('78411', tensor(0.9498, dtype=torch.float64)), ('84798', tensor(0.9491, dtype=torch.float64)), ('96220', tensor(0.9491, dtype=torch.float64)), ('82155', tensor(0.9490, dtype=torch.float64)), ('96266', tensor(0.9490, dtype=torch.float64)), ('79034', tensor(0.9490, dtype=torch.float64)), ('79319', tensor(0.9488, dtype=torch.float64)), ('78039', tensor(0.9487, dtype=torch.float64)), ('96222', tensor(0.9484, dtype=torch.float64)), ('78926', tensor(0.9481, dtype=torch.float64)), ('96286', tensor(0.9477, dtype=torch.float64)), ('79217', t

In [32]:
len(similarity_dict)
print(similarity_dict)

{'70677': tensor(0.9474, dtype=torch.float64), '70872': tensor(0.9448, dtype=torch.float64), '73684': tensor(0.9186, dtype=torch.float64), '73685': tensor(0.9364, dtype=torch.float64), '77997': tensor(0.9244, dtype=torch.float64), '78016': tensor(0.9090, dtype=torch.float64), '78031': tensor(0.9215, dtype=torch.float64), '78039': tensor(0.9487, dtype=torch.float64), '78042': tensor(0.9255, dtype=torch.float64), '78051': tensor(0.9212, dtype=torch.float64), '78158': tensor(0.9367, dtype=torch.float64), '78178': tensor(0.9250, dtype=torch.float64), '78280': tensor(0.9739, dtype=torch.float64), '78282': tensor(0.9249, dtype=torch.float64), '78367': tensor(0.9739, dtype=torch.float64), '78411': tensor(0.9498, dtype=torch.float64), '78449': tensor(0.9034, dtype=torch.float64), '78454': tensor(0.9257, dtype=torch.float64), '78456': tensor(0.9093, dtype=torch.float64), '78704': tensor(0.9153, dtype=torch.float64), '78710': tensor(0.9255, dtype=torch.float64), '78718': tensor(0.9526, dtype=tor

In [29]:
F.cosine_similarity(state2tensor(student_knowledge_state),state2tensor(other_student_knowledge_state),dim=0)

tensor(0.8931, dtype=torch.float64)

In [26]:
other_student_knowledge_state

'[-0.00000000e+00 -0.00000000e+00  0.00000000e+00 -1.67927835e-12\n  1.25752224e-04 -9.51097012e-01 -0.00000000e+00 -9.51992750e-01\n -3.35311232e-20  2.07435387e-05 -7.35780239e-01  0.00000000e+00\n  1.30255157e-05  3.14421813e-24  3.59367450e-15 -1.85063298e-10\n -9.65762511e-02  1.78139011e-18 -9.51289475e-01  5.04064523e-26\n  1.23864590e-11  3.01758264e-04  2.61251330e-06  7.04366787e-10\n  1.63676168e-06 -8.46333786e-20 -1.10111767e-04  2.36999829e-12\n -2.46272214e-13  1.90847251e-12 -1.55079353e-04  0.00000000e+00\n  9.51192141e-01  0.00000000e+00 -3.48834240e-17 -9.51978803e-01\n  1.72824740e-14 -2.67265255e-14 -3.58582474e-02 -2.07676043e-10\n -1.24718482e-03  1.67566148e-04  0.00000000e+00 -0.00000000e+00\n  2.76528450e-20 -4.23807822e-10 -9.51936126e-01  6.08820017e-08\n -0.00000000e+00  6.75263550e-07  6.62639709e-26  0.00000000e+00\n -1.85078532e-11  1.16040017e-13  7.28996419e-10 -3.01749563e-11\n  1.82104429e-06 -9.38173108e-26 -9.39041317e-01  2.40171421e-03\n  0.00000

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [20]:
type(student_knowledge_state)

numpy.ndarray

In [ ]:
[-4.21929190e-06 -4.43273716e-12  9.51992214e-01 -1.67927521e-12
  1.25752471e-04 -9.51097012e-01 -2.39947699e-19 -9.51992750e-01
 -3.35306095e-20  0.00000000e+00 -0.00000000e+00  4.08395987e-14
  1.30255030e-05  0.00000000e+00  3.59363045e-15 -1.85063659e-10
 -9.65764970e-02  1.78138680e-18 -9.51289475e-01  5.04067451e-26
  1.23865102e-11  3.01758526e-04  2.61252080e-06  0.00000000e+00
  1.63676327e-06 -8.46334626e-20 -1.10111563e-04  2.36999266e-12
 -2.46271482e-13  1.90847446e-12 -0.00000000e+00  7.06778585e-17
  9.51192141e-01  7.70749028e-17 -0.00000000e+00 -0.00000000e+00
  1.72825079e-14 -0.00000000e+00 -3.58582065e-02 -0.00000000e+00
 -1.24718598e-03  1.67565668e-04  0.00000000e+00 -1.14751722e-10
  2.76530065e-20 -4.23808488e-10 -0.00000000e+00  6.08820017e-08
 -9.51862216e-01  6.75264232e-07  0.00000000e+00  1.11119949e-16
 -1.85079486e-11  1.16039706e-13  7.28997918e-10 -3.01750118e-11
  1.82104600e-06 -9.38166760e-26 -0.00000000e+00  2.40171864e-03
  0.00000000e+00  3.11220265e-06  0.00000000e+00  0.00000000e+00
 -6.06484355e-05  8.89375806e-02  0.00000000e+00 -1.59917877e-03
  9.51992750e-01 -0.00000000e+00 -0.00000000e+00 -3.94228747e-21
  9.19129014e-01 -2.55262080e-07  6.08864514e-09  6.35964267e-11
 -0.00000000e+00  9.51992750e-01 -9.51992750e-01 -1.71055348e-07
 -9.49787585e-20  0.00000000e+00 -5.40845905e-13 -1.94090796e-18
  3.80706189e-10 -3.56801834e-18 -8.98116589e-01  8.66388891e-18
 -9.51992512e-01 -9.50979531e-01  0.00000000e+00 -2.99498900e-26
 -9.51830208e-01 -3.20669324e-10  0.00000000e+00  9.51992750e-01
  3.16771956e-11 -0.00000000e+00 -1.12128126e-10 -9.51992750e-01]